# Welcome to PYNQ Audio
This notebook shows the basic recording and playback features of the PYNQ-Z2.  
It uses the audio jack `HP+MIC` to play back recordings; it can take inputs from 
the microphone on `HP+MIC` or `LINE_IN`. Pre-recorded audio sample can also be taken
as input. Moreover, visualization with matplotlib is shown.
## Create new audio object

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
pAudio = base.audio
pAudio.select_microphone()
pAudio.set_volume(20)

## Bypass audio
Users can select either `LINE_IN`, or `HP+MIC` as the input port.
In the following example, we choose `LINE_IN`. To choose `MIC`:
```python
pAudio.select_microphone()
```
or choose `LINE_IN`:
```python
pAudio.select_line_in()
```

In [3]:
pAudio.select_microphone()

## Record and play
Record a 5-second sample and save it into a file.

In [4]:
pAudio.record(1)
pAudio.save("recording_1.wav")

In [5]:
pAudio.load("/home/xilinx/jupyter_notebooks/base/audio/recording_1.wav")
pAudio.play()

from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/base/audio/recording_1.wav")

In [6]:
temp = pAudio.buffer<<8

left = temp[0::2]
right = temp[1::2]

## Threading

In [7]:
# From Strath-SDR RFSoC QPSK example
import threading
import time
import ipywidgets as ipw
import queue

class AudioThreading():
    def __init__(self, pynq_audio, record_time, chunk, start=True):
        """
        Create new dma-based data timer.
        callback: function to call with data chunk
        gen: function to call to return data chunk
             (usually a dma channel's transfer function)
        """
        self.stopping = not start
        self.record_time = record_time
        self._buffer = queue.Queue()
        self._pynq_audio = pynq_audio
        self.chunk = chunk
        self.closed = True

    def _do(self):
        """
        Generate new data and restart timer thread.
        Should never be run directly. use `start()` instead.
        """
        while not self.stopping:
            self._pynq_audio.record(self.record_time)
            self._buffer.put(self._pynq_audio.buffer << 8)

#     def start(self):
    def __enter__(self):
        
        self._audio_interface = self._pynq_audio
        
        """ Start the data generator thread. """
        if not self.stopping:           
            self.stopping = False
            thread = threading.Thread(target=self._do)
            thread.start()
            
        self.closed = False
        return self
    
#     def stop(self):
    def __exit__(self, type, value, traceback):
        """
        Stop a running data generator thread.
        Does not need a lock, since the spawned timer thread will only read `self.stopping`.
        """
        self.closed = True
        self._buffer.put(None)
        
    # Rev.ai Code example for microphone stream:
    def generator(self):
        while not self.stopping:
            """
            Use a blocking get() to ensure there's at least one chunk of
            data, and stop iteration if the chunk is None => stop on pause!
            """
            chunk = self._buffer.get()
            if chunk is None:
                return
            data = [chunk]

            while True:
                try:
                    chunk = self._buffer.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)

# Rev.ai Microphone Stream:

## Importing Relavent Modules:

In [8]:
from rev_ai.models import MediaConfig
from rev_ai.streamingclient import RevAiStreamingClient
from PPFunctions_Live import *

## Sampling Rate of Audio Codec:

In [9]:
rate = 48 * 1000
chunk = int(rate/20000) # Approx 4s

## Personal Access Token for Rev.ai:

In [10]:
access_token = "020xN8wEvpFJ57K5xz4CjnhKVkC0kDJO74fKvx58chPRJQHUChMfrQyTWWooMnfO5H5kyGiVdHJHSroppSQFIU9g69v2E"

## Raw Microphone Input to Create Media Config:

In [11]:
example_mc = MediaConfig('audio/x-raw', 'interleaved', rate, 'S32LE', 2)
streamclient = RevAiStreamingClient(access_token, example_mc)

## Calling Thread for Speech-to-Text:

In [13]:
# with AudioThreading(pAudio, 1, chunk) as stream:
with AudioThreading(pAudio, 1, chunk) as stream:

    #present_t = time.time()
    #end_t = present_t + 15

    #if present_t < end_t:
        ## Uses try method to enable users to manually close the stream
    try:
        ## Starts the server connection and thread sending microphone audio
        print("new chunk!")
        response_gen = streamclient.start(stream.generator())

        ## Iterates through responses and prints them
        for response in response_gen:
            sub = real_t(response)
            print(sub)

    except KeyboardInterrupt:
        ## Ends the WebSocket connection.
        streamclient.end()
        pass

new chunk!
Connected, Job ID : woLKfFXO8AaOLbWv
okay
okay and we
okay and we are
okay and we are
okay and we are sure
okay and we are sure
Okay. And we are sure subtitle.
and
and have
and have
and have scottish we'd
and have scottish we'd
and have scottish we'd work
and have scottish we'd work very
and have scottish we'd work very well <laugh>
and have scottish we'd work very well <laugh>
and have scottish we'd work very well <laugh>
And have Scottish accent we'd work very well. <laugh>.


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/share/pynq-venv/lib/python3.8/site-packages/rev_ai/streamingclient.py", line 167, in _send_data
    self.client.send_binary(chunk)
  File "/usr/local/share/pynq-venv/lib/python3.8/site-packages/websocket/_core.py", line 317, in send_binary
    return self.send(payload, ABNF.OPCODE_BINARY)
  File "/usr/local/share/pynq-venv/lib/python3.8/site-packages/websocket/_core.py", line 283, in send
    return self.send_frame(frame)
  File "/usr/local/share/pynq-venv/lib/python3.8/site-packages/websocket/_core.py", line 311, in send_frame
    l = self._send(data)
  File "/usr/local/share/pynq-venv/lib/python3.8/site-packages/websocket/_core.py", line 512, in _send
    return send(self.sock, data)
  File "/usr/local/share/pynq

## Load and play
Load a sample and play the loaded sample.

In [ ]:
pAudio.load("/home/xilinx/jupyter_notebooks/base/audio/recording_1.wav")
pAudio.play()

## Play in notebook
Since the samples are in 24-bit PCM format, 
users can play the audio directly in notebook.

In [ ]:
from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/base/audio/recording_1.wav")

## Plotting PCM data

Users can display the audio data in notebook:

1. Plot the audio signal's amplitude over time.
2. Plot the spectrogram of the audio signal.

The next cell reads the saved audio file and processes it into a `numpy` array.
Note that if the audio sample width is not standard, additional processing
is required. In the following example, the `sample_width` is read from the
wave file itself (24-bit dual-channel PCM audio, where `sample_width` is 3 bytes).

In [ ]:
%matplotlib inline
import wave
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft

wav_path = "/home/xilinx/jupyter_notebooks/base/audio/recording_0.wav"
with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    
temp_buffer = np.empty((num_frames, num_channels, 4), dtype=np.uint8)
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, 
                                                    sample_width)
temp_buffer[:, :, sample_width:] = \
    (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 255
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])

### 1. Amplitude over time

In [ ]:
for channel_index in range(num_channels):
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Audio in Time Domain (Channel {})'.format(channel_index))
    plt.xlabel('Time in s')
    plt.ylabel('Amplitude')
    time_axis = np.arange(0, num_frames/sample_rate, 1/sample_rate)
    plt.plot(time_axis, frames[:, channel_index])
    plt.show()

### 2. Frequency spectrum

In [ ]:
for channel_index in range(num_channels):
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Audio in Frequency Demain (Channel {})'.format(channel_index))
    plt.xlabel('Frequency in Hz')
    plt.ylabel('Magnitude')
    temp = fft(frames[:, channel_index])
    yf = temp[1:len(temp)//2]
    xf = np.linspace(0.0, sample_rate/2, len(yf))
    plt.plot(xf, abs(yf))
    plt.show()

### 3. Frequency spectrum over time
Use the `classic` plot style for better display.

In [ ]:
for channel_index in range(num_channels):
    np.seterr(divide='ignore', invalid='ignore')
    matplotlib.style.use("classic")
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Signal Spectogram (Channel {})'.format(channel_index))
    plt.xlabel('Time in s')
    plt.ylabel('Frequency in Hz')
    plt.specgram(frames[:, channel_index], Fs=sample_rate)